In [ ]:
# 실습 목록

# 문장 임베딩(Sentence BERT)을 이용한 챗봇 구현
# Faiss와 SBERT를 이용한 시맨틱 검색기(Semantic Search)
# 임베딩 검색기(SBERT) 파인튜닝하기

## BERT를 이용한 금융 뉴스 긍정, 부정 분류
## BERT를 이용한 KorNLI 풀어보기
## BERT를 이용한 개체명인식
## BERT를 이용한 기계독해
## BERT를 이용한 다중 레이블 분류

### 문장 임베딩(Sentence BERT)을 이용한 챗봇 구현

In [4]:
#pip install sentence_transformers

import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import urllib.request
from sentence_transformers import SentenceTransformer


In [5]:
url = "https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv"
urllib.request.urlretrieve(url=url, filename="./attach/ChatBotData.csv")
train_data = pd.read_csv('./attach/ChatBotData.csv')
train_data.head()


,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [6]:
# 사전 훈련된 BERT 로드
model_name = 'xlm-r-100langs-bert-base-nli-stsb-mean-tokens'
model = SentenceTransformer('sentence-transformers/'+model_name)
    # 100가지 언어를 지원(한국어 포함)하는 다국어 BERT BASE 모델로 
    # SNLI 데이터를 학습 후 STS-B 데이터로 학습되었으며, 
    # 문장벡터를 얻기 위해 평균풀링(mean-tokens)을 사용 
    # 다시 말해서 NLI 데이터를 학습 후에 STS 데이터로 추가 파인 튜닝한 모델이라는 의미

    # < SentenceTransformer로 로드할 수 있는 다양한 모델에 대한 리스트 >
    # => https://huggingface.co/models?library=sentence-transformers


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

c:\Dev\anaconda3\envs\TORCH38_CPU\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\imim555\.cache\huggingface\hub\models--sentence-transformers--xlm-r-100langs-bert-base-nli-stsb-mean-tokens. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.86k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
# 데이터의의 질문 컬럼에 대해서 문장 임베딩값을 구하고 저장
train_data['embedding']=train_data.apply(lambda row: model.encode(row.Q), axis = 1)
# axis=0 ; 열을 하나씩 떼와서 반환 => 컬럼 데이터 1개만 읽을 수 있음 1D(Series)
# axis=1 ; 행을 하나씩 떼와서 반환 => 컬럼 데이터들을 읽을 수 있음 2D(DataFrame)

In [17]:
# 2개의 문장벡터에서 코사인 유사도 계산
def cos_sim(A,B):
    return dot(A,B)/(norm(A)*norm(B))

In [18]:
# answer함수 정의
# 임의의 질문이 들어오면 해당 질문의 문장 임베딩값과 
# 훈련용 질문 샘플의 임베딩값의 코사인유사도를 구하고
# 가장 높은 유사도값을 가진 질문 샘플과 답변샘플을 리턴
def return_answer(question):
    embedding = model.encode(question)
    train_data['score'] = train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
    return train_data.loc[train_data['score'].idxmax()]['A']


In [23]:
# 챗봇 테스트
question = '결혼하고 싶어'
res=return_answer(question)
print(res)

question = '나랑 커피 먹을래?'
res=return_answer(question)
print(res)

question = '방가방가방가'
res=return_answer(question)
print(res)

question = 'ㅋㅋㅋㅋ'
res=return_answer(question)
print(res)

좋은 사람이랑 결혼할 수 있을 거예요.
카페인이 필요한 시간인가 봐요.
네 말씀해주세요.
안타깝지만 잊어버리세요.


### Faiss와 SBERT를 이용한 시맨틱 검색기(Semantic Search)
semantic search는 기존의 키워드 매칭이 아닌 문장의 의미에 초점을 맞춘 정보검색시스템


In [29]:
# Faiss는 벡터화 된 데이터를 효율적으로 인덱싱하고 검색을 수행
# Facebook AI에서 구축한 C++기반 라이브러리
#!pip install faiss-cpu # faiss-gpu
#!pip install -U sentence-transformers

import numpy as np
import os
import pandas as pd
import urllib.request
import faiss
import time
from sentence_transformers import SentenceTransformer

In [ ]:
# 데이터 로드
# 약 100만개의 뉴스 기사 제목 데이터를 사용
url="https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/19.%20Topic%20Modeling%20(LDA%2C%20BERT-Based)/dataset/abcnews-date-text.csv"
filename="./attach/abcnews-date-text.csv"
urllib.request.urlretrieve(url=url, filename=filename)

df = pd.read_csv(filename)
df

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers
...,...,...
1082163,20170630,when is it ok to compliment a womans smile a g...
1082164,20170630,white house defends trumps tweet
1082165,20170630,winter closes in on tasmania as snow ice falls
1082166,20170630,womens world cup australia wins despite atapat...


In [34]:
# 리스트 형태 변환
data = df.headline_text.to_list()
print('총 샘플의 개수 :', len(data))
data[:5]



총 샘플의 개수 : 1082168


['aba decides against community broadcasting licence',
 'act fire witnesses must be aware of defamation',
 'a g calls for infrastructure protection summit',
 'air nz staff in aust strike for pay rise',
 'air nz strike to affect australian travellers']

In [35]:
# SBERT 임베딩
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
encoded_data = model.encode(data)
print('임베딩 된 벡터 수 :', len(encoded_data))
encoded_data



modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

c:\Dev\anaconda3\envs\TORCH38_CPU\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\imim555\.cache\huggingface\hub\models--sentence-transformers--distilbert-base-nli-mean-tokens. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.80k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
# 인덱스 정의 및 데이터 추가
# ------------------------------
# faiss.IndexFlatIP(768) : 
#   - 인덱스 인스턴스 생성성
#   - 768 차원의 내적(Inner Product, IP) 기반 벡터 검색
#   - 가장 기본적인 유사도 검색 방법
# faiss.IndexIDMap(...) : 
#   - 벡터마다 사용자 지정 ID를 부여
#   - default는 ID 없이 순서대로 인덱싱됨(벡터위치 기반)
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))

# 인덱스에 임베딩된 데이터 추가
index.add_with_ids(encoded_data, np.array(range(0,len(data)))) 

In [ ]:
# 검색 및 시간 측정
# 유사도가 높은 상위 5개 샘플 추출
def search(query):
    t = time.time()
    query_vector = model.decode([query])
    k = 5 
    top_k = index.search(query_vector, k)
    print('total time: {}'.format(time.time()-t))
    return [data[_id] for _id in top_k[1].tolist()[0]]


In [ ]:
#query = str(input())
query = 'Underwater Forest Discovered'
results = search(query)

print('results:')
for result in results:
    print('\t', result)

### 임베딩 검색기(SBERT) 파인튜닝하기
- 한국어 SBERT모델인 BGE-M3을 커스텀 데이터로 파인튜닝하여 검색성능을 올려보자
- 위 모델은 RAG 등에 많이 사용됨

In [ ]:
# !pip : Python 패키지 관리자(pip)를 사용하여 패키지 설치
# -U : 업데이트 옵션(이미 설치된 경우 최신 버전으로 업데이트)
# == : 특정 버전의 패키지 지정

# FlagEmbedding	1.2.11	텍스트 임베딩 모델 라이브러리. OpenAI, BGE(Base-Guided Embeddings) 모델 등 다양한 임베딩 모델을 지원.
# transformers	4.44.2	Hugging Face의 NLP 모델 라이브러리. GPT, BERT, Llama 등 다양한 모델을 로드하고 사용할 수 있음.
# peft	0.13.0	파라미터 효율적 미세 조정(PEFT, Parameter-Efficient Fine-Tuning) 라이브러리. LLaMA, GPT 등의 LLM을 효과적으로 미세 조정할 때 사용.
# faiss-gpu	1.7.2	벡터 검색 라이브러리. GPU 가속을 지원하며, 대규모 벡터 검색(Nearest Neighbor Search)에 사용됨.
# LM_Cocktail	0.0.4	RAG(Retrieval-Augmented Generation) 모델을 쉽게 구현하기 위한 라이브러리. LLM과 검색 시스템을 결합하는 데 도움.

!pip install -U FlagEmbedding==1.2.11 transformers==4.44.2 peft==0.13.0 faiss-gpu==1.7.2 LM_Cocktail==0.0.4

In [ ]:
# 데이터 로드
# -----------------------------------
# 데이터셋의 형식은 반드시 jsonl 형태
# 각 데이터는 "query", "pos", "neg"로 구성되며, 
# 이때 "query"는 검색어, 
# "pos"는 해당 검색어와 연관된 문서들이 포함된 리스트, 
# "neg"는 해당 검색어와 연관되지 않은 문서들이 포함된 리스트

import json

# 번역된 데이터
translated_data = [
    {"query": "다섯 명의 여성이 해변을 따라 플립플롭을 신고 걸어간다.", "pos": ["플립플롭을 신은 몇몇 여성들이 해변을 따라 걸어가고 있다"], "neg": ["4명의 여성이 해변에 앉아 있다.", "1996년에 개혁이 있었다.", "그녀는 자신의 기록을 정정하기 위해 법정에 가지 않을 것이다.", "그 남자는 하와이에 대해 이야기하고 있다.", "한 여성이 밖에 서 있다.", "전투는 끝났다.", "한 무리의 사람들이 배구를 하고 있다."]},
    {"query": "한 여성이 높은 절벽 위에서 한 발로 서서 강을 내려다보고 있다.", "pos": ["한 여성이 절벽 위에 서 있다."], "neg": ["한 여성이 의자에 앉아 있다.", "조지 부시는 공화당원들에게 최고 고문들의 조언에 반하여 이 어리석은 생각을 고려조차 하지 않겠다고 말했다.", "그 가족은 무너지고 있었다.", "아무도 회의에 나타나지 않았다", "한 소년이 밖에서 모래를 가지고 놀고 있다.", "전보를 받자마자 끝났다.", "한 아이가 자기 방에서 책을 읽고 있다."]},
    {"query": "두 여성이 악기를 연주하고 있다; 한 명은 클라리넷, 다른 한 명은 바이올린을 연주한다.", "pos": ["몇 사람이 곡을 연주하고 있다."], "neg": ["두 여성이 기타와 드럼을 연주하고 있다.", "한 남자가 산을 스키를 타고 내려가고 있다.", "살인자가 생각했던 때에 치명적인 용량이 투여되지 않았다.", "자전거를 타고 있는 사람", "그 소녀는 아치길에 기대어 서 있다.", "한 무리의 여성들이 소파 오페라를 보고 있다.", "사람들은 나이가 들어도 절대 잊지 않는다."]},
    {"query": "파란색 탱크톱을 입은 소녀가 앉아서 세 마리의 개를 지켜보고 있다.", "pos": ["한 소녀가 파란색을 입고 있다."], "neg": ["한 소녀가 세 마리의 고양이와 함께 있다.", "사람들이 장례 행렬을 지켜보고 있다.", "그 아이는 검은색을 입고 있다.", "공립학교에서 우리에게 재정은 문제이다.", "수영장에 있는 아이들.", "폭행당하는 것은 진정시키는 일이다.", "나는 18살에 심각한 문제에 직면했다."]},
    {"query": "노란 개가 숲길을 따라 달리고 있다.", "pos": ["개가 달리고 있다"], "neg": ["고양이가 달리고 있다", "스틸은 그녀의 원래 이야기를 지키지 않았다.", "이 규칙은 사람들이 자녀 양육비를 내는 것을 막는다.", "조끼를 입은 남자가 차 안에 앉아 있다.", "검은 옷을 입고 흰색 반다나와 선글라스를 낀 사람이 버스 정류장에서 기다리고 있다.", "글로브나 메일 중 어느 쪽도 캐나다의 현재 도로 체계 상태에 대해 언급하지 않았다.", "스프링 크릭 시설은 오래되고 구식이다."]},
    {"query": "각 단계에서의 필수 활동과 그 활동들과 관련된 중요한 요소들을 설명한다.", "pos": ["필수 활동에 대한 중요 요소들이 설명되어 있다."], "neg": ["중요한 활동들을 설명하지만 그 활동들과 관련된 중요한 요소들에 대한 규정은 없다.", "사람들이 항의하기 위해 모여 있다.", "주 정부는 당신이 그렇게 하기를 선호할 것이다.", "한 소녀가 한 소년 옆에 앉아 있다.", "두 남성이 공연하고 있다.", "아무도 뛰고 있지 않다", "콘라드는 머리를 맞도록 음모를 꾸미고 있었다."]},
    {"query": "한 남자가 레스토랑에서 연설을 하고 있다.", "pos": ["한 사람이 연설을 하고 있다."], "neg": ["그 남자는 테이블에 앉아 음식을 먹고 있다.", "이것은 확실히 승인이 아니다.", "그들은 은퇴 때문에 집을 팔았지, 대출 때문이 아니다.", "미주리 주의 인장은 완벽하다.", "누군가가 손을 들고 있다.", "한 운동선수가 1500미터 수영 경기에 참가하고 있다.", "두 남자가 마술 쇼를 보고 있다."]},
    {"query": "인디언들이 코트를 입고 음식과 음료를 가지고 모임을 갖고 있다.", "pos": ["인디언 그룹이 음식과 음료를 가지고 모임을 갖고 있다"], "neg": ["인디언 그룹이 장례식을 하고 있다", "이것은 팔마의 큰 투우장에서 겨울 오후에만 공연된다.", "올바른 정보는 법률 서비스 관행과 사법 체계를 강화할 수 있다.", "한편, 본토는 인구가 없었다.", "두 아이가 자고 있다.", "어부가 원숭이를 잡으려고 하고 있다", "사람들이 기차 안에 있다"]},
    {"query": "보라색 머리를 한 여성이 밖에서 자전거를 타고 있다.", "pos": ["한 여성이 자전거를 타고 있다."], "neg": ["한 여성이 공원에서 조깅을 하고 있다.", "그 거리는 하얀색으로 칠해진 집들로 가득했다.", "한 그룹이 안에서 영화를 보고 있다.", "소풍에서 남자들이 스테이크를 자르고 있다", "여러 명의 요리사들이 앉아서 음식에 대해 이야기하고 있다.", "위원회는 중요한 대안들이 고려되지 않았다고 지적한다.", "우리는 장작이 다 떨어져서 불을 위해 소나무 바늘을 사용해야 했다."]},
    {"query": "한 남자가 도시 거리에서 인력거로 두 여성을 끌고 있다.", "pos": ["한 남자가 도시에 있다."], "neg": ["한 남자가 비행기 조종사이다.", "그것은 지루하고 평범하다.", "아침 햇살이 밝게 비치고 따뜻했다.", "두 사람이 부두에서 뛰어내렸다.", "사람들이 우주선 발사를 보고 있다.", "테레사 수녀는 쉬운 선택이다.", "원하는 속도로 갈 수 있는 것은 가치가 있다."]}
]

# JSONL 파일로 저장
with open('./attach/toy_finetune_data.jsonl', 'w', encoding='utf-8') as f:
    for item in translated_data:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')

print("데이터가 './attach/toy_finetune_data.jsonl' 파일로 저장되었습니다.")


In [ ]:
# Negative Sampling
# 데이터 구축이 잘 되어있지 않을 때 사용
# 보유 데이터에서 랜덤으로 neg를 뽑는 방법
!python -m FlagEmbedding.baai_general_embedding.finetune.hn_mine \
--model_name_or_path BAAI/bge-m3 \
--input_file toy_finetune_data.jsonl \
--output_file toy_finetune_data_minedHN.jsonl \
--range_for_sampling 2-200 \
--negative_number 15 \
--use_gpu_for_searching 


In [ ]:
# 임베딩 유사도 값 확인
from FlagEmbedding import FlagModel

sentences_1 = ["다섯 명의 여성이 해변을 따라 플립플롭을 신고 걸어간다."]
sentences_2 = ["플립플롭을 신은 몇몇 여성들이 해변을 따라 걸어가고 있다", "꽁꽁 얼어붙은 한강 위로 고양이가 걸어가고 있다"]


In [ ]:
# use_fp16의 값을 True로 사용하면 약간의 성능 저하는 있지만 속도를 더 빠르게 할 수 있습니다.
model = FlagModel('BAAI/bge-m3', use_fp16=True)
fine_tuned_model = FlagModel('/content/checkpoint', use_fp16=True)

# 기존 모델로 각각 임베딩
embeddings_1 = model.encode(sentences_1)
embeddings_2 = model.encode(sentences_2)

# 기존 모델로부터 나온 임베딩으로 유사도 계산
similarity_from_base_model = embeddings_1 @ embeddings_2.T

# 파인 튜닝 모델로 각각 임베딩
embeddings_1 = fine_tuned_model.encode(sentences_1)
embeddings_2 = fine_tuned_model.encode(sentences_2)

# 파인 튜닝 모델로부터 나온 임베딩으로 유사도 계산
similarity_from_fine_tuned_model = embeddings_1 @ embeddings_2.T

print('기존 모델:', similarity_from_base_model)
print('파인 튜닝된 모델:', similarity_from_fine_tuned_model)


In [ ]:
# 모델 병합
# 파인 튜닝 모델과 기존 모델을 병합하여
# 일반화 성능 유지+파인 튜닝 모델 성능 개선
from LM_Cocktail import mix_models, mix_models_with_data

model = mix_models(
    model_names_or_paths=["BAAI/bge-m3", './attach/content/checkpoint'],
    model_type = 'encoder',
    weights=[0.5, 0.5], # 가중치 조절
    output_path = './attach/mixed_model_1'
)

In [ ]:
# 병합 모델 로딩
sentences_1 = ["다섯 명의 여성이 해변을 따라 플립플롭을 신고 걸어간다."]
sentences_2 = ["플립플롭을 신은 몇몇 여성들이 해변을 따라 걸어가고 있다", "꽁꽁 얼어붙은 한강 위로 고양이가 걸어가고 있다"]

# Setting use_fp16 to True speeds up computation with a slight performance degradation
model = FlagModel('./attach/mixed_model_1', use_fp16=True)

embeddings_1 = model.encode(sentences_1)
embeddings_2 = model.encode(sentences_2)
similarity = embeddings_1 @ embeddings_2.T
    # @ 연산자는 행렬곱
    # embeddings_1 (N × D) × embeddings_2.T (D × M) => N x M

print('융합된 모델:', similarity)
